# Этап 10: Knowledge Distillation (Дистилляция знаний)

Дистилляция — это способ передать знания от большой и точной модели (Teacher) к маленькой и быстрой (Student). Студент учится не только на правильных ответах (hard targets), но и на «уверенности» учителя (soft targets).

### Идея метода:
Учитель говорит: «Это с вероятностью 90% символ 'A' и 9% символ 'B'». Студент пытается повторить именно это распределение вероятностей, что дает ему гораздо больше информации, чем просто буква 'A'.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.model import GPTLanguageModel, device, get_batch

# 1. Загружаем Учителя (уже обученная модель)
teacher = GPTLanguageModel().to(device)
teacher.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
teacher.eval() # Учитель всегда в режиме eval

# 2. Создаем Студента (модель поменьше)
# В реальной жизни мы бы уменьшили n_layer или n_embd в src.model.py
# Для примера просто создадим новую копию и будем ее учить
student = GPTLanguageModel().to(device)
student.train()

def distillation_loss(student_logits, teacher_logits, labels, T=2.0, alpha=0.5):
    # Soft loss (KL Divergence)
    soft_targets = F.softmax(teacher_logits / T, dim=-1)
    soft_prob = F.log_softmax(student_logits / T, dim=-1)
    distillation_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean') * (T**2)
    
    # Hard loss (обычный Cross Entropy)
    student_loss = F.cross_entropy(student_logits.view(-1, student_logits.size(-1)), labels.view(-1))
    
    return alpha * distillation_loss + (1 - alpha) * student_loss

optimizer = torch.optim.AdamW(student.parameters(), lr=1e-4)

# Цикл дистилляции (одна итерация)
xb, yb = get_batch('train')
with torch.no_grad():
    teacher_logits, _ = teacher(xb)

student_logits, _ = student(xb)
loss = distillation_loss(student_logits, teacher_logits, yb)

optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Loss дистилляции: {loss.item():.4f}")